In [103]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import math
from jupyterthemes import jtplot
jtplot.style(theme='monokai')
jtplot.style(context='talk', fscale=1.4, spines=False, gridlines='--')
jtplot.style(ticks=True, grid=False, figsize=(10, 10))
import datetime
from psaw import PushshiftAPI
from __future__ import annotations
import os
import json

In [107]:

def get_names(path: str = '' ) -> dict[str, list[str]]:
    """
    Reads the character data base characters.json.
    """
    with open(os.path.join(path, "characters.json"), "r") as f:
        characters = json.load(f)
    return characters

def find_names(doc: str, name_db: dict[str, list[str]]) -> list[str]:
    """
    Returns a list of all characters that were present in the document.
    Characters are returned as full names and not as they were found.
    """
    out = list()
    for name, keys in name_db.items():
        for k in keys:
            if k in doc:
                out.append(name)
                break
    return out


In [108]:
api = PushshiftAPI()
features = ["title", "id", "score", "author", "created_utc", "selftext"]
subreddit = "asoiaf"
date1 = int(datetime.datetime(2015,7,16).timestamp())
date2 = int(datetime.datetime(2019,4,14).timestamp())
gen = api.search_submissions(subreddit = subreddit, after = date1, before = date2, q='', filter = features, limit = 3 * 10**5)

In [81]:
results = list(gen)

/home/oskar/.local/lib/python3.9/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
/home/oskar/.local/lib/python3.9/site-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [83]:

ids, title, score, author, created_utc, selftext = [], [], [], [], [], []
for i in range(len(results)):
    try:
        selftext.append(results[i].d_['selftext'])
        ids.append(i)
        title.append(results[i].d_['title'])
        score.append(results[i].d_['score'])
        created_utc.append(results[i].d_['created_utc'])
        author.append(results[i].d_['author'])
    except KeyError:
        pass
        

data = pd.DataFrame(
{
    "id" : ids,
    "title" : title,
    "score" : score,
    "created" : created_utc,
    "author" : author,
    "selftext" : selftext
}, index = None)

In [118]:
# To see amount of posts that acutally has text in selftext
def get_usefull_data():
    text_list = [text for text in data['selftext'] if text != '' ]
    text_list = [text for text in text_list if text != '[removed]' ]
    return len(text_list)

def get_postw_names():
    text_list = [text for text in data['names'] if text != [] ]
    return len(text_list)
get_usefull_data()

72090

In [116]:
def mentioned_chars(df: pd.DataFrame) -> pd.DataFrame:
    names = get_names()
    row_name_finder = lambda row: find_names(row["selftext"], names)
    
    df["names"] = df.apply(row_name_finder, axis=1)
    return df
test = mentioned_chars(data)
test

,id,title,score,created,author,selftext,names
0,0,How Game of Thrones Lost Interest in Tyrion La...,1,1555192642,Euphoric_Translator,,[]
1,1,(Spoilers Extended) The meeting that I'm waiti...,376,1555191033,JRR_STARK,,[]
2,2,Unborn Lannister,1,1555191006,kcconk,[removed],[]
3,3,(Spoilers extended) This sub is going to explo...,7,1555189915,seperationsunday,2019. r/asoiaf is about to E.X.P.L.O.D.E\n\nAr...,[]
4,4,(Spoilers extended) For all those that love to...,1,1555188219,KingPoTheThird,,[]
...,...,...,...,...,...,...,...
117737,118855,(Spoilers All) Need help finding Stannis quote...,1,1436999414,user1444,Can you guys list every scene you can think of...,[Stannis Baratheon]
117738,118856,(Spoilers All) I didn't realize how much fores...,62,1436999159,Compeau,This is from a Bran chapter in AGOT.\n\n&gt;He...,"[Robert Baratheon, Jon Snow, Sansa Stark, Arya..."
117739,118857,Doesn't Tyrion have any bastards?,1,1436998933,[deleted],,[]
117740,118858,(Spoilers All) AGOT Sansa chapter with the for...,0,1436998367,itsmearyastark,"I don't know if this has been mentioned here, ...","[Jon Snow, Sansa Stark]"


In [119]:
get_postw_names()

40991

In [120]:
data.to_csv('asioaf_data.csv',index=False)